<div align="center">
<p align="center" style="width: 100%;">
    <img src="https://raw.githubusercontent.com/vlm-run/.github/refs/heads/main/profile/assets/vlm-black.svg" alt="VLM Run Logo" width="80" style="margin-bottom: -5px; color: #2e3138; vertical-align: middle; padding-right: 5px;"><br>
</p>
<p align="center"><a href="https://docs.vlm.run"><b>Website</b></a> | <a href="https://docs.vlm.run/"><b>API Docs</b></a> | <a href="https://docs.vlm.run/blog"><b>Blog</b></a> | <a href="https://discord.gg/AMApC2UzVY"><b>Discord</b></a> | <a href="https://chat.vlm.run"><b>Chat</b></a>
</p>
</div>

# VLM Run Orion - Protein Structure Interpretation

This notebook demonstrates how to use [VLM Run Orion's](https://vlm.run/orion) vision capabilities to analyze 3D ribbon diagrams of proteins and identify key secondary structural elements and overall fold architecture. This is particularly useful for structural biology research, identifying alpha-helices, beta-sheets, loops, and classifying protein folds.

For more details on the API, see the [Agent API docs](https://docs.vlm.run/agents/introduction).

## Prerequisites

- Python 3.10+
- VLM Run API key (get one at [app.vlm.run](https://app.vlm.run))
- VLM Run Python Client with OpenAI extra `vlmrun[openai]`


## Setup

First, install the required packages and configure the environment.


In [8]:
# Install required packages
%pip install vlmrun[openai] --upgrade --quiet
%pip install cachetools pillow requests numpy --quiet


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import os
import getpass

VLMRUN_API_KEY = os.getenv("VLMRUN_API_KEY", None)
if VLMRUN_API_KEY is None:
    VLMRUN_API_KEY = getpass.getpass("Enter your VLM Run API key: ")


## Initialize the VLM Run Client

We use the OpenAI-compatible chat completions interface through the VLM Run SDK.


In [10]:
from vlmrun.client import VLMRun

BASE_URL = os.getenv("VLMRUN_BASE_URL", "https://agent.vlm.run/v1")
client = VLMRun(api_key=VLMRUN_API_KEY, base_url=BASE_URL)
print("VLM Run client initialized successfully!")
print(f"Base URL: {BASE_URL}")


VLM Run client initialized successfully!
Base URL: https://agent.vlm.run/v1


## Response Models

We define Pydantic models for structured outputs. The response will include secondary structure elements, structural context, and fold classification.


In [11]:
from pydantic import BaseModel, Field, field_validator
from typing import List, Optional, Literal


class AlphaHelix(BaseModel):
    """Information about an alpha-helix."""
    count: int = Field(..., description="Number of alpha-helices identified")
    orientation: str = Field(..., description="Relative orientation description (e.g., 'bundle of four parallel helices')")
    spatial_location: str = Field(..., description="Spatial description of helix location (e.g., 'exterior surface', 'central core')")


class BetaSheet(BaseModel):
    """Information about a beta-sheet."""
    configuration: Literal["Parallel", "Anti-parallel", "Mixed"] = Field(..., description="Configuration of the beta-sheet")
    number_of_strands: int = Field(..., description="Estimated number of strands in the sheet")
    spatial_location: str = Field(..., description="Spatial description of sheet location")


class LoopRegion(BaseModel):
    """Information about loops and turns."""
    description: str = Field(..., description="Description of significant disordered regions or sharp turns (e.g., beta-turns)")
    location: str = Field(..., description="Location of the loop/turn region")


class StructuralContext(BaseModel):
    """Structural context information."""
    n_terminus_location: Optional[str] = Field(None, description="Location of N-terminus if clearly identifiable")
    c_terminus_location: Optional[str] = Field(None, description="Location of C-terminus if clearly identifiable")
    ligand_binding_sites: List[str] = Field(default_factory=list, description="Description of any visible ligand binding sites or prosthetic groups (e.g., heme group)")
    spatial_topology: str = Field(..., description="Spatial description of protein topology (e.g., 'large beta-barrel dominates the central core')")

    @field_validator("ligand_binding_sites", mode="before")
    def _ensure_ligand_binding_sites(cls, v):
        if v is None:
            return []
        return v


class ProteinStructureResponse(BaseModel):
    """Response containing the protein structure analysis."""
    alpha_helices: AlphaHelix = Field(..., description="Information about alpha-helices in the protein")
    beta_sheets: List[BetaSheet] = Field(..., description="List of beta-sheets identified")
    loops_and_turns: List[LoopRegion] = Field(default_factory=list, description="List of significant loops and turns")
    structural_context: StructuralContext = Field(..., description="Structural context including termini and binding sites")
    fold_classification: Optional[str] = Field(None, description="Fold classification if recognizable (e.g., TIM barrel, Rossmann fold)")
    structural_summary: str = Field(..., description="Overall summary of the structural components")

print("Response models defined successfully!")


Response models defined successfully!


## Helper Functions

We create helper functions to simplify making chat completion requests with structured outputs.


In [12]:
import hashlib
import json
from typing import Any, Type, TypeVar

import cachetools
from vlmrun.common.image import encode_image
from PIL import Image


T = TypeVar('T', bound=BaseModel)


def custom_key(prompt: str, image_path: str | None = None, response_model: Type[T] | None = None, model: str = "vlmrun-orion-1:auto"):
    """Custom key for caching chat_completion."""
    response_key = hashlib.sha256(json.dumps(response_model.model_json_schema(), sort_keys=True).encode()).hexdigest() if response_model else ""
    image_key = hashlib.sha256(image_path.encode()).hexdigest() if image_path else ""
    return (prompt, image_key, response_key, model)


@cachetools.cached(cache=cachetools.TTLCache(maxsize=100, ttl=3600), key=custom_key)
def chat_completion(
    prompt: str,
    image_path: str | None = None,
    response_model: Type[T] | None = None,
    model: str = "vlmrun-orion-1:auto"
) -> tuple[BaseModel | str, str]:
    """
    Make a chat completion request with structured output for protein structure interpretation.

    Args:
        prompt: The prompt describing the protein structure analysis task
        image_path: Path to the image file containing the 3D ribbon diagram
        response_model: Pydantic model for structured output
        model: Model to use (default: vlmrun-orion-1:auto)

    Returns:
        Tuple of (parsed response model or text, session_id)
    """
    content = [{"type": "text", "text": prompt}]
    
    # Add image if provided
    if image_path:
        image = Image.open(image_path)
        image_data = encode_image(image, format="JPEG")
        content.append({"type": "image_url", "image_url": {"url": image_data}})

    kwargs = {
        "model": model,
        "messages": [{"role": "user", "content": content}]
    }

    if response_model:
        kwargs["response_format"] = {
            "type": "json_schema",
            "schema": response_model.model_json_schema()
        }

    response = client.agent.completions.create(**kwargs)
    response_text = response.choices[0].message.content
    session_id = response.session_id

    if response_model:
        result = response_model.model_validate_json(response_text)
        return result, session_id

    return response_text, session_id

print("Helper functions defined!")


Helper functions defined!


## Protein Structure Interpretation

Analyze a 3D ribbon diagram of a protein to identify secondary structural elements and overall fold architecture. Provide the path to an image file containing the protein structure diagram.


In [13]:
# Combined prompt for protein structure interpretation
PROTEIN_STRUCTURE_PROMPT = """
Task: Analyze the provided 3D ribbon diagram of a protein and identify its key secondary structural elements and overall fold architecture.

Instructions:

Secondary Structure Localization: 
* α-helices: Identify and count the prominent α-helices. Note their relative orientation (e.g., "bundle of four parallel helices").
* β-sheets: Locate β-pleated sheets. Specify if they are arranged in parallel or anti-parallel configurations and estimate the number of strands per sheet.
* Loops and Turns: Point out significant disordered regions or sharp turns (e.g., β-turns) connecting the primary motifs.

Structural Context:
* Identify the N-terminus and C-terminus if they are clearly labeled or distinguishable by color gradients.
* Describe any visible ligand binding sites or prosthetic groups (e.g., a heme group) if present in the rendering.

Spatial Description: Use the visual orientation of the ribbon (e.g., "The large β-barrel dominates the central core," or "Three α-helices are located on the exterior surface") to describe the protein's topology.

Output Format: Provide a summary of the structural components followed by a detailed breakdown of the fold classification (e.g., TIM barrel, Rossmann fold) if recognizable.
"""

print("Protein structure interpretation prompt prepared!")
print(f"\nPrompt length: {len(PROTEIN_STRUCTURE_PROMPT)} characters")


Protein structure interpretation prompt prepared!

Prompt length: 1233 characters


In [ ]:
# Example: Analyze protein structure
# Replace 'path/to/your/protein_structure.jpg' with the actual path to your image
image_path = "ribbon.jpg"

# Analyze the protein structure
result, session_id = chat_completion(
    prompt=PROTEIN_STRUCTURE_PROMPT,
    image_path=image_path,
    response_model=ProteinStructureResponse,
    model="vlmrun-orion-1:auto"
)

print(">> RESPONSE")
print(result)
print(f"\n>> SESSION ID: {session_id}")

print("\n>> STRUCTURAL SUMMARY")
print("=" * 80)
print(result.structural_summary)
print("=" * 80)

print("\n>> ALPHA-HELICES")
print("=" * 80)
print(f"Count: {result.alpha_helices.count}")
print(f"Orientation: {result.alpha_helices.orientation}")
print(f"Spatial Location: {result.alpha_helices.spatial_location}")
print("=" * 80)

print("\n>> BETA-SHEETS")
print("=" * 80)
if result.beta_sheets:
    for i, sheet in enumerate(result.beta_sheets, 1):
        print(f"\nBeta-Sheet #{i}:")
        print(f"  Configuration: {sheet.configuration}")
        print(f"  Number of Strands: {sheet.number_of_strands}")
        print(f"  Spatial Location: {sheet.spatial_location}")
else:
    print("No beta-sheets identified.")
print("=" * 80)

if result.loops_and_turns:
    print("\n>> LOOPS AND TURNS")
    print("=" * 80)
    for i, loop in enumerate(result.loops_and_turns, 1):
        print(f"\nLoop/Turn #{i}:")
        print(f"  Description: {loop.description}")
        print(f"  Location: {loop.location}")
    print("=" * 80)

print("\n>> STRUCTURAL CONTEXT")
print("=" * 80)
if result.structural_context.n_terminus_location:
    print(f"N-terminus: {result.structural_context.n_terminus_location}")
if result.structural_context.c_terminus_location:
    print(f"C-terminus: {result.structural_context.c_terminus_location}")
if result.structural_context.ligand_binding_sites:
    print("Ligand Binding Sites:")
    for site in result.structural_context.ligand_binding_sites:
        print(f"  - {site}")
print(f"Topology: {result.structural_context.spatial_topology}")
print("=" * 80)

if result.fold_classification:
    print("\n>> FOLD CLASSIFICATION")
    print("=" * 80)
    print(result.fold_classification)
    print("=" * 80)


>> RESPONSE
alpha_helices=AlphaHelix(count=6, orientation='arranged in a cluster surrounding the central beta-sheet core', spatial_location='exterior layers packing against the surface of the central beta-sheet') beta_sheets=[BetaSheet(configuration='Anti-parallel', number_of_strands=5, spatial_location='central core of the protein structure')] loops_and_turns=[LoopRegion(description='Flexible ribbon segments including sharp turns connecting the secondary motifs', location='Interspersed between the ends of the beta-strands and alpha-helices')] structural_context=StructuralContext(n_terminus_location=None, c_terminus_location=None, ligand_binding_sites=[], spatial_topology='A central core dominated by a five-stranded anti-parallel beta-sheet flanked by six alpha-helices on its exterior surfaces') fold_classification='Alpha/beta sandwich fold' structural_summary='The protein exhibits a classic alpha/beta architecture consisting of a central core of five anti-parallel beta-strands forming

---

## Conclusion

This notebook demonstrated how to use **VLM Run Orion** to analyze 3D ribbon diagrams of proteins and identify secondary structural elements and overall fold architecture.

### Key Takeaways

1. **Structured Prompts**: Well-structured prompts with clear instructions for secondary structure localization, structural context, and spatial description produce better results.
2. **Session Management**: Use `session_id` to track your analysis requests.
3. **Image Input**: Provide clear, high-quality images of protein ribbon diagrams for best results.
4. **Structured Output**: The model returns detailed information about alpha-helices, beta-sheets, loops, structural context, and fold classification in a structured format.

### Next Steps

- Experiment with different types of protein structures (enzymes, membrane proteins, etc.)
- Try analyzing proteins with different fold classifications
- Explore the [VLM Run Documentation](https://docs.vlm.run) for more capabilities
- Join our [Discord community](https://discord.gg/AMApC2UzVY) for support

Happy analyzing! 🧬
